## <span style="color:#ff5f27">👨🏻‍🏫 Build Index </span>

In this notebook you will create a feature group for your candidate embeddings.

In [1]:
import time

# Start the timer
notebook_start_time = time.time()

## <span style="color:#ff5f27">📝 Imports </span>

In [2]:
import tensorflow as tf
import pprint
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [3]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://snurran.hops.works/p/17527
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27">🎯 Compute Candidate Embeddings </span>

You start by computing candidate embeddings for all items in the training data.

First, you load your candidate model. Recall that you uploaded it to the Hopsworks Model Registry in the previous notebook. If you don't have the model locally you can download it from the Model Registry using the following code:

In [4]:
model = mr.get_model(
    name="candidate_model",
    version=1,
)
model_path = model.download()

If you already have the model saved locally you can simply replace `model_path` with the path to your model.

In [5]:
candidate_model = tf.saved_model.load(model_path)

Next you compute the embeddings of all candidate items that were used to train the retrieval model.

In [6]:
feature_view = fs.get_feature_view(
    name="retrieval", 
    version=1,
)

In [7]:
train_df, val_df, test_df, _, _, _ = feature_view.train_validation_test_split(
    validation_size=0.1, 
    test_size=0.1,
    description='Retrieval dataset splits',
)

Finished: Reading data from Hopsworks, using ArrowFlight (307.72s) 


In [8]:
train_df.head(3)

,customer_id,article_id,t_dat,price,month_sin,month_cos,age,club_member_status,age_group,garment_group_name,index_group_name
0,e7f61a6cf67eb0380b5dc595b3e666c3200ea369bb9831...,751994007,1598918400000,0.050831,-1.0,-1.836970e-16,40.0,ACTIVE,36-45,Trousers Denim,Menswear
1,5e06d8dd0e1f2c69f83cf872c02371bdffde0b36d21020...,659462002,1541203200000,0.088966,-0.5,8.660254e-01,33.0,ACTIVE,26-35,Outdoor,Ladieswear
2,5b0ab511aeff7ea7424db992c119d19a8036853c2d35b3...,894320003,1599696000000,0.033881,-1.0,-1.836970e-16,54.0,ACTIVE,46-55,Knitwear,Ladieswear


In [9]:
# Get the list of input features for the candidate model from the model schema
model_schema = model.model_schema['input_schema']['columnar_schema']
candidate_features = [feat['name'] for feat in model_schema]

# Select the candidate features from the training DataFrame
item_df = train_df[candidate_features]

# Drop duplicate rows based on the 'article_id' column to get unique candidate items
item_df.drop_duplicates(subset="article_id", inplace=True)

item_df.head(3)

,article_id,garment_group_name,index_group_name
0,751994007,Trousers Denim,Menswear
1,659462002,Outdoor,Ladieswear
2,894320003,Knitwear,Ladieswear


In [10]:
# Create a TensorFlow dataset from the item DataFrame
item_ds = tf.data.Dataset.from_tensor_slices(
    {col: item_df[col] for col in item_df})

# Compute embeddings for all candidate items using the candidate_model
candidate_embeddings = item_ds.batch(2048).map(
    lambda x: (x["article_id"], candidate_model(x))
)

> Strictly speaking, you haven't actually computed the candidate embeddings yet, as the dataset functions are lazily evaluated.

## <span style="color:#ff5f27">⚙️ Data Preparation </span>


In [11]:
# Concatenate all article IDs and embeddings from the candidate_embeddings dataset
all_article_ids = tf.concat([batch[0] for batch in candidate_embeddings], axis=0)
all_embeddings = tf.concat([batch[1] for batch in candidate_embeddings], axis=0)

# Convert tensors to numpy arrays
all_article_ids_np = all_article_ids.numpy().astype(int)
all_embeddings_np = all_embeddings.numpy()

# Convert numpy arrays to lists
items_ids_list = all_article_ids_np.tolist()
embeddings_list = all_embeddings_np.tolist()

In [12]:
# Create a DataFrame
data_emb = pd.DataFrame({
    'article_id': items_ids_list, 
    'embeddings': embeddings_list,
})

data_emb.head()

,article_id,embeddings
0,751994007,"[20.64307403564453, -2.8355846405029297, 3.350..."
1,659462002,"[18.738597869873047, -3.4859068393707275, 4.53..."
2,894320003,"[17.060224533081055, -4.006529331207275, 3.308..."
3,833021002,"[21.060016632080078, -2.525385618209839, 1.854..."
4,687338001,"[20.22753143310547, -2.479985237121582, 3.4194..."


## <span style="color:#ff5f27">🪄 Feature Group Creation </span>

Now you are ready to create a feature group for your candidate embeddings.

To begin with, you need to create your Embedding Index where you will specify the name of the embeddings feature and the embeddings length.
Then you attach this index to the FG.

In [13]:
from hsfs import embedding

# Create the Embedding Index
emb = embedding.EmbeddingIndex()

emb.add_embedding(
    "embeddings",                           # Embeddings feature name
    len(data_emb["embeddings"].iloc[0]),    # Embeddings length
)

In [14]:
# Get or create the 'candidate_embeddings_fg' feature group
candidate_embeddings_fg = fs.get_or_create_feature_group(
    name="candidate_embeddings_fg",
    embedding_index=emb,                    # Specify the Embedding Index
    primary_key=['article_id'],
    version=1,
    description='Embeddings for each article',
    online_enabled=True,
)

candidate_embeddings_fg.insert(data_emb)

Feature Group created successfully, explore it at 
https://snurran.hops.works/p/17527/fs/17475/fg/20592


Uploading Dataframe: 0.00% |          | Rows 0/69226 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: candidate_embeddings_fg_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://snurran.hops.works/p/17527/jobs/named/candidate_embeddings_fg_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f2c60154730>, None)

## <span style="color:#ff5f27">🪄 Feature View Creation </span>


In [15]:
# Get or create the 'candidate_embeddings' feature view
feature_view = fs.get_or_create_feature_view(
    name="candidate_embeddings",
    version=1,
    description='Embeddings of each article',
    query=candidate_embeddings_fg.select(["article_id"]),
)

Feature view created successfully, explore it at 
https://snurran.hops.works/p/17527/fs/17475/fv/candidate_embeddings/version/1


---

In [16]:
# End the timer
notebook_end_time = time.time()

# Calculate and print the execution time
notebook_execution_time = notebook_end_time - notebook_start_time
print(f"⌛️ Notebook Execution time: {notebook_execution_time:.2f} seconds")

⌛️ Notebook Execution time: 324.91 seconds


---
## <span style="color:#ff5f27">⏩️ Next Steps </span>

At this point you have a recommender system that is able to generate a set of candidate items for a customer. However, many of these could be poor, as the candidate model was trained with only a few subset of the features. In the next notebook, you'll create a ranking dataset to train a *ranking model* to do more fine-grained predictions.